In [2]:
import pandas as pd
import optuna


from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

2024-11-30 23:14:17.145466: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 23:14:17.596849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 23:14:17.598774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 23:14:18.628165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
X_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_train.csv", header=None)
X_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/X_test.csv", header=None)
y_train = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_train.csv", header=None)
y_test = pd.read_csv("/home/pavithra/projects/spam_or_ham_fsec/data/target/y_test.csv", header=None)



print("X train ",X_train.shape)
print("X test",X_test.shape)
print("Y_Train", y_train.shape)
print("Y_test", y_test.shape)

X train  (3417, 1000)
X test (375, 1000)
Y_Train (3417, 1)
Y_test (375, 1)


In [4]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=1000))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               128128    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 136449 (533.00 KB)
Trainable params: 136449 (533.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:

def create_model(trial):
    """
    Create and return the model I wanna try on.
    Which is a nueral network with one hidden layer and a dropout layer.
    """
    # Set the hyperparameters which I wanna try.
    layer1_units = trial.suggest_int('layer1_units', 64, 256, step=32)
    layer2_units = trial.suggest_int('layer2_units', 32, 128, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    model = Sequential()
    model.add(Dense(layer1_units, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(dropout_rate))
    model.add(Dense(layer2_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

def objective(trial):
    model = create_model(trial)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=10,
                        batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]),
                        verbose=0)

    # Evaluate on validation set
    val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
    return val_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best hyperparameters:", study.best_params)
print(f"Best accuracy: {study.best_value}")

[I 2024-11-30 23:14:21,149] A new study created in memory with name: no-name-f0854266-ac31-45f6-a0c8-98e889743c9e
/tmp/ipykernel_199153/1146100160.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
[I 2024-11-30 23:14:28,170] Trial 0 finished with value: 11.979991912841797 and parameters: {'layer1_units': 256, 'layer2_units': 32, 'dropout_rate': 0.39682595076410143, 'learning_rate': 0.00010376141143771641, 'batch_size': 16}. Best is trial 0 with value: 11.979991912841797.
/tmp/ipykernel_199153/1146100160.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('l

Best hyperparameters: {'layer1_units': 256, 'layer2_units': 64, 'dropout_rate': 0.33382076889331297, 'learning_rate': 0.00010281328736417759, 'batch_size': 64}
Best accuracy: 2.1616101264953613


In [7]:
Best hyperparameters: {'layer1_units': 128, 'layer2_units': 64, 'dropout_rate': 0.3874414297981386, 'learning_rate': 0.00018557740326790697, 'batch_size': 32}
Best accuracy: 13.9380521774292

SyntaxError: invalid syntax (2923225420.py, line 1)

In [1]:
best_trial = study.best_params
final_model = create_model(optuna.trial.FixedTrial(best_trial))
final_model.fit(X_train, y_train, epochs=20, batch_size=best_trial['batch_size'])

NameError: name 'study' is not defined